In [ ]:

import cv2
import numpy as np
import tensorflow as tf
import pandas as pds
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D, Cropping2D
from keras.layers.pooling import MaxPooling2D
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

def _normalize_grayscale_bak(image_data):
    '''
    Normalize the data features to the variable X_normalized
    '''
    a = -0.5
    b = 0.5
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

def _normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = 0.1
    b = 0.9
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

def _cropping_image(image_data):
    """
    50 rows pixels from the top of the image
    20 rows pixels from the bottom of the image
    0 columns of pixels from the left of the image
    0 columns of pixels from the right of the image
    (height, width, channels)
    """
    img_roi_x      = 50  
    img_roi_y      = 0  
    img_roi_height = image_data.shape[0] - 50 - 20
    img_roi_width  = image_data.shape[1]                                                                       
    return image_data[img_roi_x:(img_roi_x+img_roi_height),img_roi_y:(img_roi_y+img_roi_width)]

def image_preprocess(file_path):
    '''
    '''
    # 1. load image
    origan_image = cv2.imread(file_path.strip())
    # 2. turn image to gray
    gray_image = cv2.cvtColor(origan_image, cv2.COLOR_BGR2GRAY)
    # 3. cropping image
    cropped_image = _cropping_image(gray_image)
    # 5. normalize image
    normalize_image = _normalize_grayscale(cropped_image)
    # 6. reshape
    image_size = normalize_image.shape
    reshape_image = np.reshape(normalize_image, (image_size[0], image_size[1], 1) )
    #print(reshape_image.shape)
    return reshape_image
    
def data_generator(csv_file, batch_size=80, left_camera_compl=0.2, right_camera_compl=-0.2):
    """
    Image generator. Returns batches of images indefinitely
    - path : path to csv file
    - batch_size : batch size
    """
    csv_data = pds.read_csv(csv_file)
    
    data_left_camera = csv_data['left']
    data_center_camera = csv_data['center']
    data_right_camera = csv_data['right']
    data_steering = csv_data['steering']
    print(csv_data.shape)
    
    corrent_index = 0
    features = []
    labels = []
    
    while 1 :
        
        if corrent_index >= len(csv_data) :
            corrent_index = 0
            
        images_left_camera = image_preprocess(data_left_camera[corrent_index])
        images_center_camera = image_preprocess(data_center_camera[corrent_index])
        images_right_camera = image_preprocess(data_right_camera[corrent_index])
        steering_angle = data_steering[corrent_index]
        
        features.append(images_left_camera)
        labels.append(steering_angle+left_camera_compl)
        features.append(np.fliplr(images_left_camera))
        labels.append(-(steering_angle+left_camera_compl))
        features.append(images_center_camera)
        labels.append(steering_angle)
        features.append(np.fliplr(images_center_camera))
        labels.append(-(steering_angle))
        features.append(images_right_camera)
        labels.append(steering_angle+right_camera_compl)
        features.append(np.fliplr(images_right_camera))
        labels.append(-(steering_angle+right_camera_compl))
        
        corrent_index += 1;
        
        if(len(features) >= batch_size):
            #print(np.array(features).shape, np.array(labels).shape)
            yield (np.array(features), np.array(labels))
            features = []
            labels = []
            
def create_keras_model(feature_shape=(90,320,1)):
    '''
    
    '''
    model = Sequential()
    #
    #model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=feature_shape)) 
    #
    model.add(Conv2D(24, 5, 5, activation='relu', input_shape=feature_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    #
    model.add(Conv2D(36, 5, 5, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    #
    model.add(Conv2D(48, 5, 5, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    #
    model.add(Conv2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    #
    #model.add(Conv2D(64, 3, 3, activation='relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    #
    model.add(Flatten())
    #
    model.add(Dense(1164, activation='relu'))
    model.add(Dropout(0.5))
    #
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.5))
    #
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))
    #
    model.add(Dense(10, activation='relu'))
    model.add(Dropout(0.5))
    #
    model.add(Dense(1, activation='linear'))

    return model
    
def train_model(data_generator):
    '''
    CarND-Behavioral-Cloning-P3
    '''
    bhvcln_model = create_keras_model()
    print(bhvcln_model.summary)
    
    bhvcln_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    
    # train model
    bhvcln_model.fit_generator(data_generator, samples_per_epoch=24000, nb_epoch=20)
    
if __name__ == '__main__':
    data_gene = data_generator('driving_log.csv')
    train_model(data_gene)
